Dataset Link : https://www.kaggle.com/datasets/jessicali9530/kuc-hackathon-winter-2018

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"prajaktaghumatkar99","key":"92322a3a568aebb0499d100a0ca14275"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jessicali9530/kuc-hackathon-winter-2018

 61% 25.0M/40.7M [00:00<00:00, 47.0MB/s]
100% 40.7M/40.7M [00:00<00:00, 70.4MB/s]


In [ ]:
from zipfile import ZipFile
file_name="kuc-hackathon-winter-2018.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from wordcloud import WordCloud
from textblob import TextBlob
from nltk.corpus import stopwords
from collections import Counter
import warnings; warnings.simplefilter('ignore')
import nltk
import re
import string
from string import punctuation
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from xgboost import XGBClassifier
from lightgbm import LGBMModel,LGBMClassifier, plot_importance

!pip install catboost
from catboost import CatBoostClassifier


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
train = pd.read_csv('drugsComTrain_raw.csv')
test = pd.read_csv('drugsComTest_raw.csv')
train

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10,31-May-15,125
161293,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1,1-Nov-11,34
161294,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2,15-Mar-14,35
161295,47128,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10,19-Sep-15,79


In [ ]:
# as both the dataset contains same columns we can combine them for better analysis

data = pd.concat([train, test])
data.head()

In [ ]:
# describing the data

data.describe()

In [ ]:
# taking out information from the data

data.info()

In [ ]:
# Dropping the data points with null values 
data = data.dropna(how = 'any', axis = 0)
# lowercasing the column names so it will be easier for access ^^
data.columns = data.columns.str.lower()
# Sorting the dataframe
data.sort_values(['rating'], ascending = True, inplace = True)
data.reset_index(drop = True, inplace = True)
# Converting the date in to date time format 
data['date'] = pd.to_datetime(data['date'])
data.head(10)

In [ ]:
# Total unique conditions in the dataset
print (data['condition'].nunique(), "\n")

print ("some of the conditions are : ", data['condition'].unique()[0:10])

In [ ]:
#top 10 drugs with rating equals 10
data.loc[data['rating'] == 10, :]['drugname'].value_counts().head(10)

In [ ]:
# Word cloud of the reviews with rating equal to 10
#stopwords = set(STOPWORDS)

df_rate_ten = data.loc[data.rating == 10, 'review']
text = (' '.join(df_rate_ten))

wordcloud = WordCloud(width = 1000, height = 500, background_color = 'white').generate(text)
plt.figure(figsize=(15, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off');

In [ ]:
# This barplot shows the top 20 drugs with the 10/10 rating

# Setting the Parameter
sns.set(font_scale = 1.2, style = 'darkgrid')
plt.rcParams['figure.figsize'] = [15, 8]

rating = dict(data.loc[data.rating == 10, "drugname"].value_counts())
drugname = list(rating.keys())
drug_rating = list(rating.values())

sns_rating = sns.barplot(x = drugname[0:20], y = drug_rating[0:20])

sns_rating.set_title('Top 20 drugs with 10/10 rating')
sns_rating.set_ylabel("Number of Ratings")
sns_rating.set_xlabel("Drug Names")
plt.setp(sns_rating.get_xticklabels(), rotation=90);

In [ ]:
# This barplot show the top 10 conditions the people are suffering.
cond = dict(data['condition'].value_counts())
top_condition = list(cond.keys())[0:10]
values = list(cond.values())[0:10]
sns.set(style = 'darkgrid', font_scale = 1.3)
plt.rcParams['figure.figsize'] = [18, 7]

sns_ = sns.barplot(x = top_condition, y = values, palette = 'winter')
sns_.set_title("Top 10 conditions")
sns_.set_xlabel("Conditions")
sns_.set_ylabel("Count");

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# A countplot of the ratings so we can see the distribution of the ratings
plt.rcParams['figure.figsize'] = [20,8]
sns.set(font_scale = 1.4, style = 'darkgrid')
fig, ax = plt.subplots(1, 2)

sns_1 = sns.countplot(data['rating'], palette = 'spring', order = list(range(10, 0, -1)), ax = ax[0])
sns_2 = sns.distplot(data['rating'], ax = ax[1])
sns_1.set_title('Count of Ratings')
sns_1.set_xlabel("Rating")

sns_2.set_title('Distribution of Ratings')
sns_2.set_xlabel("Rating")

**Preprocessing**

In [ ]:
# Giving the Sentiment according to the ratings
import math
data['sentiment_rate'] = data['rating'].apply(lambda x: math.ceil(x/2))
data['sentiment_rate'].value_counts()

Word count plot

In [ ]:
def review_clean(review): 
    # changing review to lower cases.
    lower = review.str.lower()
    
    # Replacing the repeating pattern of &#039;
    pattern_remove = lower.str.replace("&#039;", "")
    
    # Removing all the special Characters
    special_remove = pattern_remove.str.replace(r'[^\w\d\s]',' ')
    
    # Removing all the non ASCII characters
    ascii_remove = special_remove.str.replace(r'[^\x00-\x7F]+',' ')
    
    # Removing the leading and trailing Whitespaces
    whitespace_remove = ascii_remove.str.replace(r'^\s+|\s+?$','')
    
    # Replacing multiple Spaces with Single Space
    multiw_remove = whitespace_remove.str.replace(r'\s+',' ')
    
    # Replacing Two or more dots with one
    dataframe = multiw_remove.str.replace(r'\.{2,}', ' ')
    
    return dataframe

In [ ]:
# Contraction Dictionary for the expansion

contractions_dict = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because",
    "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not",
    "doesn’t": "does not", "don't": "do not", "don’t": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not",
    "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
    "i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
    "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not","might've": "might have",
    "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
    "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
    "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", 
    "shouldn't've": "should not have", "so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have",
    "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "they'd": "they would",
    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
    "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
    "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
    "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
    "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
    "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
    "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y’all": "you all", "y'all'd": "you all would",
    "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have",
    "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "ain’t": "am not", "aren’t": "are not",
    "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not", "couldn’t’ve": "could not have",
    "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
    "hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have",
    "he’s": "he is", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
    "i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would", "it’d’ve": "it would have",
    "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us", "ma’am": "madam", "mayn’t": "may not",
    "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
    "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock",
    "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have",
    "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "she’s": "she is",
    "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have", "so’s": "so is",
    "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would", "there’d’ve": "there would have",
    "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will", "they’ll’ve": "they will have",
    "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have",
    "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
    "what’ll’ve": "what will have", "what’re": "what are", "what’s": "what is", "what’ve": "what have", "when’s": "when is",
    "when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will",
    "who’ll’ve": "who will have", "who’s": "who is", "who’ve": "who have","why’s": "why is", "why’ve": "why have", "will’ve": "will have",
    "won’t": "will not", "won’t’ve": "will not have", "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have",
    "y’all": "you all", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have", "y’all’re": "you all are",
    "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
    "you’re": "you are", "you’re": "you are", "you’ve": "you have"
}
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function expand the contractions if there's any
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [ ]:
#applying the above assumptions into a column
data['review_clean'] = review_clean(data['review'])

# Expanding the contractions
data['review_clean'] = data['review_clean'].apply(lambda x: expand_contractions(x))

# Removing punctuations
data['review_clean'] = data['review_clean'].apply(lambda x: ''.join(word for word in x if word not in punctuation))

In [ ]:
data.head()

In [ ]:
from textblob import TextBlob
"""I have used textblob module to give the sentiment polarity of the review. This polarity is given to both the cleaned and uncleaned review"""
#from nltk.corpus import stopwords
from collections import Counter
import warnings; warnings.simplefilter('ignore')
import nltk
import string
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

# Removing the stopwords from the review_clean column
stop_words = set(stopwords.words('english'))
data['review_clean'] = data['review_clean'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

# Removing the word stems using the Snowball Stemmer
Snow_ball = SnowballStemmer("english")
data['review_clean'] = data['review_clean'].apply(lambda x: " ".join(Snow_ball.stem(word) for word in x.split()))

# Separating the day, month and year from the Date

data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year

data.head()

In [ ]:
def sentiment(review):
    # Sentiment polarity of the reviews
    pol = []
    for i in review:
        analysis = TextBlob(i)
        pol.append(analysis.sentiment.polarity)
    return pol

In [ ]:
data['sentiment'] = sentiment(data['review'])

data['sentiment_clean'] = sentiment(data['review_clean'])

np.corrcoef(data['sentiment'], data['rating'])

np.corrcoef(data['sentiment_clean'], data['rating'])

Now we can see that the correlation coefficient between setiment of the uncleaned review included with rating is higher than with the cleaned review, so for better results lets try the cleaning without removing the stopwords and the snowball stemmer.


In [ ]:
# Cleaning the reviews without removing the stop words and using snowball stemmer

data['review_clean_ss'] = review_clean(data['review'])

data['review_clean_ss'] = data['review_clean_ss'].apply(lambda x: expand_contractions(x))

data['review_clean_ss'] = data['review_clean_ss'].apply(lambda x: ''.join(word for word in x if word not in punctuation))

data['sentiment_clean_ss'] = sentiment(data['review_clean_ss'])

np.corrcoef(data['sentiment_clean_ss'], data['rating'])

In [ ]:
data.head()

In [ ]:
# Calculating weighted ratings
# Formula : (R*v + C*m)/(v+m)
# R - data['rating']
# v - data['usefulcount']
# C (mean rating for entire dataset) - data['rating'].mean()
# m (least # of votes/usefulcount required to be accepted in 70 percentile) - data['usefulcount'].quantile(0.70)

# Calculating C
C=int(data['rating'].mean())

#Calculating m
m = data['usefulcount'].quantile(0.70)
m

In [ ]:
data['weighted_rating'] = ((data['rating']*data['usefulcount']) + (C*m))/(data['usefulcount']+m)

In [ ]:
data.head()

In [ ]:
#Word count in each review
data['count_word']=data["review_clean_ss"].apply(lambda x: len(str(x).split()))

#Unique word count 
data['count_unique_word']=data["review_clean_ss"].apply(lambda x: len(set(str(x).split())))

#Letter count
data['count_letters']=data["review_clean_ss"].apply(lambda x: len(str(x)))

#punctuation count
data["count_punctuations"] = data["review"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

#upper case words count
data["count_words_upper"] = data["review"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

#title case words count
data["count_words_title"] = data["review"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

#Number of stopwords
data["count_stopwords"] = data["review"].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

#Average length of the words
data["mean_word_len"] = data["review_clean_ss"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

data['mean_word_len'] = data['mean_word_len'].fillna((data['mean_word_len'].mean()))

In [ ]:
data.columns

In [ ]:
# Correlation Heatmap of the features engineered
plt.rcParams['figure.figsize'] = [17,15]
sns.set(font_scale = 1.2)
corr = data.select_dtypes(include = 'int64').corr()
sns_ = sns.heatmap(corr, annot = True, cmap = 'YlGnBu')
plt.setp(sns_.get_xticklabels(), rotation = 45);


+Correlation Heatmap is plotted using seaborn which contains all the new features engineered and the old features.

In [ ]:
# Label Encoding Drugname and Conditions
label_encoder_feat = {}
for feature in ['drugname', 'condition']:
    label_encoder_feat[feature] = LabelEncoder()
    data[feature] = label_encoder_feat[feature].fit_transform(data[feature])

The Label Encoder is used to change the categorical values of Drug Names and the conditions in to numerical values for the machine learning modelling. There are 3,667 unique drugs in the dataset that's why One hot encoder is not used as it would generate 3,667 new features and it would be very computationally expensive.

In [ ]:
#Print plot overviews of the first 5 movies.
data['review_clean_ss'].head()

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
data['review_clean_ss'] = data['review_clean_ss'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(data['review_clean_ss'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

In [ ]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[5000:5010]

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim.shape

In [ ]:
cosine_sim[1]

**Modelling**

In [ ]:
# Defining Features and splitting the data as train and test set

features = data[['condition', 'usefulcount', 'sentiment', 'day', 'month', 'year',
                   'sentiment_clean_ss', 'count_word', 'count_unique_word', 'count_letters',
                   'count_punctuations', 'count_words_upper', 'count_words_title',
                   'count_stopwords', 'mean_word_len']]

target = data['sentiment_rate']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.25, random_state = 42)
print ("The Train set size ", X_train.shape)
print ("The Test set size ", X_test.shape)

Model - I LightGBM

In [ ]:
# Training Model - I

clf = LGBMClassifier(
        n_estimators=10000,
        learning_rate=0.10,
        num_leaves=30,
        subsample=.9,
        max_depth=7,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=2,
        silent=-1,
        verbose=-1,
        )
model = clf.fit(X_train, y_train)

# Predictions
predictions = model.predict(X_test)
print ("The Accuracy of the model on testing data is : ", accuracy_score(y_test, predictions), '\n')
print ("The confusion Matrix is \n")
print (confusion_matrix(y_test, predictions), '\n')

In [ ]:
predictions1 = model.predict(X_train)
print ("The Accuracy of the model on training data is : ", accuracy_score(y_train, predictions1), '\n')

In [ ]:
#print ("**********TRAINING SCORE********** : \n"classification_report(y_train, predictions),'\n')
#print ("**********TESTING SCORE********** : \n"classification_report(y_test, predictions))
print (classification_report(y_test, predictions))

In [ ]:
# Feature Importance Plot using LGBM
plt.rcParams['figure.figsize'] = [12, 9]
sns.set(style = 'darkgrid', font_scale = 1.2)
plot_importance(model);

In [ ]:
from sklearn import metrics
auc = metrics.roc_auc_score(y_test, predictions)
#roc_auc_score(y_score=np_pred, y_true=np_label, multi_class="ovr",average=None)

false_positive_rate, true_positive_rate, thresolds = metrics.roc_curve(y_test, predictions)

plt.figure(figsize=(10, 8), dpi=100)
plt.axis('scaled')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.title("AUC & ROC Curve")
plt.plot(false_positive_rate, true_positive_rate, 'g')
plt.fill_between(false_positive_rate, true_positive_rate, facecolor='lightgreen', alpha=0.7)
plt.text(0.95, 0.05, 'AUC = %0.4f' % auc, ha='right', fontsize=12, weight='bold', color='blue')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test,predictions, multi_class='ovr')

Model-4 Support Vector Machine (SVM)


In [ ]:
from sklearn.svm import SVC

In [ ]:
model_svc=SVC()
model_svm=model_svc.fit(X_train,y_train)

In [ ]:
# Predictions
predictions_4 = model_svm.predict(X_test)
print ("The Accuracy of the model on testing data is : ", accuracy_score(y_test, predictions_4), '\n')
predictions_41 = model_svm.predict(X_train)
print ("The Accuracy of the model on training data is : ", accuracy_score(y_train, predictions_41), '\n')
print ("The confusion Matrix is \n")
print (confusion_matrix(y_test, predictions_4), '\n')

In [ ]:
predictions_41 = model_svm.predict(X_train)
print ("The Accuracy of the model on training data is : ", accuracy_score(y_train, predictions_4), '\n')

Model-V LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
LR = LogisticRegression(solver="liblinear").fit(X_train, y_train)
predictions_5 = LR.predict(X_test)
roc_auc = roc_auc_score(y_test,preds, multi_class='ovr')
print(roc_auc) 

Model - II XGBClassifier

In [ ]:
# Training Model - II

xgb_clf = XGBClassifier(n_estimator = 10000,
                    learning_rate=0.10,
                    num_leaves=30)

model_xgb = xgb_clf.fit(X_train, y_train)

# Predictions
predictions_2 = model_xgb.predict(X_test)
print ("The Accuracy of the model on testing data is : ", accuracy_score(y_test, predictions_2), '\n')
predictions_21 = model_xgb.predict(X_train)
print ("The Accuracy of the model on training data is : ", accuracy_score(y_train, predictions_21), '\n')
print ("The confusion Matrix is \n")
print (confusion_matrix(y_test, predictions_2), '\n')

In [ ]:
#print ("**********TRAINING SCORE********** : '\n'"classification_report(y_train, predictions_2),'\n')
#print ("**********TESTING SCORE********** : '\n'"classification_report(y_test, predictions_2))
print (classification_report(y_test, predictions_2))

In [ ]:
# Feature Importance Plot using XGBClassifier
from xgboost import plot_importance # plot_importance for xgboost
plt.rcParams['figure.figsize'] = [12, 9]
plot_importance(model_xgb);

Model - III CatBoostClassifier

In [ ]:
# Training Model - III

cat_clf = CatBoostClassifier(iterations = 100,
                            learning_rate = 0.5);

model_cat = cat_clf.fit(X_train, y_train);

In [ ]:
# Predictions
predictions_3 = model_cat.predict(X_test)
print ("The Accuracy of the model on testing data is : ", accuracy_score(y_test, predictions_3), '\n')

predictions_31 = model_cat.predict(X_train)
print ("The Accuracy of the model on training data is : ", accuracy_score(y_train, predictions_31), '\n')
print ("The confusion Matrix is \n")
print (confusion_matrix(y_test, predictions_3), '\n')

print (classification_report(y_test, predictions_3))

In [ ]:
print ("**********TRAINING SCORE********** : '\n'"classification_report(y_test, predictions),'\n')
print ("**********TESTING SCORE********** : '\n'"classification_report(y_test, predictions_3))

 K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1000)
classifier.fit(X_train, y_train)

In [ ]:
predictions_4 = classifier.predict(X_test)
print ("The Accuracy of the model is : ", accuracy_score(y_test, predictions_4), '\n')

print ("The confusion matrix of train data is: "confusion_matrix(y_train, predictions_4), '\n')
print ("The confusion matrix of test data is: "confusion_matrix(y_test, predictions_4))

In [ ]:
print ("**********TRAINING SCORE********** : '\n'"classification_report(y_train, predictions_4),'\n')
print ("**********TESTING SCORE********** : '\n'"classification_report(y_test, predictions_4))

Support Vector Machine (SVM)


In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
predictions_5 = clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("The Accuracy of the model is:",metrics.accuracy_score(y_test, predictions_5),'\n')
print (confusion_matrix(y_train, predictions_5), '\n')
print (confusion_matrix(y_test, predictions_5))

In [ ]:
print (classification_report(y_train, predictions_5),'\n')
print (classification_report(y_test, predictions_5))

Support Vector Machine (SVM)


In [ ]:
X_train = np.nan_to_num(X_train)

# import SVC classifier
from sklearn.svm import SVC


# import metrics to compute accuracy
from sklearn.metrics import accuracy_score


# instantiate classifier with default hyperparameters
svc=SVC() 




# fit classifier to training set
svc.fit(X_train,y_train)


# make predictions on test set
y_pred=svc.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))


In [ ]:
print (confusion_matrix(y_test, y_pred), '\n')

print (classification_report(y_test, y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
(data.values < 0).any()

Model-5 Naive Bayes Classifier

In [ ]:
# train a Gaussian Naive Bayes classifier on the training set
from sklearn.naive_bayes import GaussianNB


# instantiate the model
gnb = GaussianNB()


# fit the model
model_gnb=gnb.fit(X_train, y_train)

In [ ]:
predictions_5 = model_gnb.predict(X_test)
print ("The Accuracy of the testing model is : ", accuracy_score(y_test, predictions_5), '\n')
predictions_51 = model_gnb.predict(X_train)
print ("The Accuracy of the model on training data is : ", accuracy_score(y_train, predictions_51), '\n')
print ("The confusion Matrix is \n")
print (confusion_matrix(y_test, predictions_5), '\n')

print (classification_report(y_test, predictions_5))

In [ ]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, predictions_5)))